In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#  biblioteca numpy que é usada para trabalhar com arrays e matrizes multidimensionais
import numpy as np 
#  biblioteca pandas que é usada para trabalhar com dados tabulares e séries temporais 
import matplotlib.pyplot as plt
#  biblioteca matplotlib que é usada para plotar gráficos 
import pandas as pd
# biblioteca pandas_datareader que é usada para importar dados do Yahoo Finance
import pandas_datareader as web
# biblioteca datetime que é usada para trabalhar com datas e horas 
import datetime as dt

# biblioteca keras que é usada para criar e treinar redes neurais
from sklearn.preprocessing import MinMaxScaler
# Sequential é uma pilha de camadas onde cada camada tem exatamente um tensor de entrada e um tensor de saída 
from tensorflow.keras.models import Sequential
# Dense é uma camada de neurônios totalmente conectados, onde cada neurônio está conectado a todos os neurônios na próxima camada
# Dropout é uma técnica de regularização que previne o overfitting 
# LSTM é uma camada de rede neural recorrente que é usada para processar sequências de dados 
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [2]:
banco = 'BBAS3.SA' # Banco do Brasil

inicio = dt.datetime(2012,1,1) # data de inicio
fim = dt.datetime(2022,11,18) # data de hoje

dados = web.DataReader(banco, 'yahoo', inicio, fim) # pega os dados do yahoo finance

In [ ]:
print(dados)

In [5]:
# Preparação dos dados
normalizando = MinMaxScaler(feature_range=(0,1)) # normalizando os dados entre 0 e 1
dados_normalizados = normalizando.fit_transform(dados['Close'].values.reshape(-1,1)) # normalizando os dados de fechamento 

previsao_dias = 3 # dias para prever

x_treinar, y_treinar = [], []

# pegando os dados de fechamento e colocando em x_treinar e y_label
for x in range(previsao_dias, len(dados_normalizados)):
    x_treinar.append(dados_normalizados[x-previsao_dias:x, 0])
    y_treinar.append(dados_normalizados[x, 0])

x_treinar, y_treinar = np.array(x_treinar), np.array(y_treinar) # transformando em array numpy 
x_treinar = np.reshape(x_treinar, (x_treinar.shape[0], x_treinar.shape[1], 1)) # redimensionando os dados

In [ ]:
print(x_treinar.shape)
print(y_treinar.shape)

In [8]:
# construindo o modelo de rede neural LSTM

modelo = Sequential()

# camada de entrada e primeira camada oculta com 50 neurônios e 60 dias de previsão 
modelo.add(LSTM(units=50, return_sequences=True, input_shape=(x_treinar.shape[1], 1))) 
# camada de dropout para evitar overfitting 
modelo.add(Dropout(0.2)) # camada de dropout para evitar overfitting 
# 50 neurônios, return_sequences=True para retornar a sequência, input_shape=(x_treinar.shape[1], 1) para pegar a quantidade de dias e 1 para pegar o valor de fechamento
modelo.add(LSTM(units=50, return_sequences=True)) 
# camada de dropout para evitar overfitting 
modelo.add(Dropout(0.2)) 
# 50 neurônios, return_sequences=True para retornar a sequência
modelo.add(LSTM(units=50))
# camada de dropout para evitar overfitting
modelo.add(Dropout(0.2))
# camada de saída com 1 neurônio
modelo.add(Dense(units=1)) 

In [ ]:
# compilando o modelo com o otimizador adam que é um otimizador estocástico de gradiente descendente
modelo.compile(optimizer='adam', loss='mean_squared_error')

modelo.fit(x_treinar, y_treinar, epochs=25, batch_size=32)

In [ ]:
print(modelo.summary())

In [12]:
# Testando o modelo com os dados de teste do futuro

# preparndo alguns dados para testar o modelo
teste_inicial = dt.datetime(2021,1,1) # data de inicio
teste_final = dt.datetime.now() # data de hoje

dados_teste = web.DataReader(banco, 'yahoo', teste_inicial, teste_final) # pega os dados do yahoo finance
precos_reais = dados_teste['Close'].values # pega os valores de fechamento

total_dados = pd.concat((dados['Close'], dados_teste['Close']), axis=0) # concatena os dados de treino e teste

modelo_entradas = total_dados[len(total_dados) - len(dados_teste) - previsao_dias:].values # pega os dados de entrada
modelo_entradas = modelo_entradas.reshape(-1,1) # redimensiona os dados
modelo_entradas = normalizando.transform(modelo_entradas) # normaliza os dados


In [14]:
print(modelo_entradas)

[[0.53083804]
 [0.51318267]
 [0.51035784]
 [0.58592279]
 [0.57674201]
 [0.59675139]
 [0.63229758]
 [0.63771188]
 [0.62241053]
 [0.63088512]
 [0.58498115]
 [0.5828625 ]
 [0.55555554]
 [0.54684558]
 [0.53248587]
 [0.51388886]
 [0.50564972]
 [0.49411484]
 [0.47292846]
 [0.49552731]
 [0.51412431]
 [0.49811677]
 [0.50823919]
 [0.50282489]
 [0.50918083]
 [0.50588509]
 [0.50047079]
 [0.49835214]
 [0.50800374]
 [0.49693977]
 [0.49999997]
 [0.49552731]
 [0.48634653]
 [0.48399243]
 [0.46916198]
 [0.3797081 ]
 [0.41737289]
 [0.41454801]
 [0.39453858]
 [0.3613465 ]
 [0.35687384]
 [0.38206215]
 [0.38347457]
 [0.40819211]
 [0.42137477]
 [0.3884181 ]
 [0.38347457]
 [0.40489641]
 [0.40842748]
 [0.40725047]
 [0.40230699]
 [0.40419023]
 [0.42372883]
 [0.4176083 ]
 [0.42372883]
 [0.42584748]
 [0.39971752]
 [0.38700564]
 [0.39689264]
 [0.3992467 ]
 [0.40254235]
 [0.42419961]
 [0.41784371]
 [0.40607346]
 [0.40513182]
 [0.39665723]
 [0.39218457]
 [0.38629945]
 [0.38818269]
 [0.39665723]
 [0.39665723]
 [0.39

In [15]:
# fazendo as previsões

x_teste = []

for x in range(previsao_dias, len(modelo_entradas)):
    x_teste.append(modelo_entradas[x-previsao_dias:x, 0])

x_teste = np.array(x_teste) # transformando em array numpy
x_teste = np.reshape(x_teste, (x_teste.shape[0], x_teste.shape[1], 1)) # redimensionando os dados

previsao_precos = modelo.predict(x_teste) # fazendo as previsões
previsao_precos = normalizando.inverse_transform(previsao_precos) # desnormalizando os dados 

15/15 [==============================] - 1s 2ms/step


In [ ]:
print(previsao_precos)

In [17]:
print(x_teste.shape)

(471, 3, 1)


In [ ]:
# representação gráfica dos dados

plt.plot(precos_reais, color='red', label=f'Preço real {banco}')
plt.plot(previsao_precos, color='green', label=f'Preço previsto {banco}')
plt.title(f'{banco} Preço real x Preço previsto')
plt.xlabel('Tempo')
plt.ylabel('Preço do Ação')
plt.legend()
plt.show()

In [19]:
# pegando os dados de fechamento e colocando em x_treinar e y_treinar para prever os próximos dias
dados_reais = [modelo_entradas[len(modelo_entradas) + 1 - previsao_dias:len(modelo_entradas+1), 0]] 
dados_reais = np.array(dados_reais) # transformando em array numpy
dados_reais = np.reshape(dados_reais, (dados_reais.shape[0], dados_reais.shape[1], 1)) # redimensionando os dados

In [20]:
print(dados_reais.shape)

(1, 2, 1)


In [21]:
previsao = modelo.predict(dados_reais) # fazendo as previsões
previsao = normalizando.inverse_transform(previsao) # desnormalizando os dados

1/1 [==============================] - 1s 921ms/step


In [22]:
print(f'Preço previsto para amanhã: {previsao}')

Preço previsto para amanhã: [[27.89208]]
